# About this Notebook
Hi fellow Kagglers,  
I have alreadt written a detailed EDA and Baseline model kernel [here](https://www.kaggle.com/manabendrarout/detailed-eda-baseline-model-plant-pathology-21).  
Do check it out before this kernel. That will set the context for what I am doing in this kernel and Why am I doing it that way.  

This kernel will mainly focus on model creation and getting a decent score. Contrary to the starter Kernel above, we will use Pytorch here just because of the simplicity with prototyping and iterations.

# Imports

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
# Asthetics
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# General
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
from tqdm import tqdm
from collections import defaultdict

# Visialisation
from PIL import Image
import cv2

# Pre Processing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Metric
from sklearn.metrics import f1_score, accuracy_score

# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

# Augmentation
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

In [ ]:
RANDOM_SEED = 42

In [ ]:
def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything()

As you all might be aware, the original image resolutions are very high; thus require lot of time for training and iterating.  
Thanks to Ankur Singh for resizing the images which speeds up the times by upto 7x, thus we will use [his dataset](https://www.kaggle.com/ankursingh12/resized-plant2021) in this kernel. If you like this Kernel, please make sure to support him as well because it is due to his efforts we are able to train this fast here.

In [ ]:
data_path = '../input/plant-pathology-2021-fgvc8'

labels_file_path = os.path.join(data_path, 'train.csv')
sample_submission_path = os.path.join(data_path, 'sample_submission.csv')

In [ ]:
df = pd.read_csv(labels_file_path)
sub_df = pd.read_csv(sample_submission_path)

In [ ]:
le = LabelEncoder()
le.fit(df['labels']);
df['labels'] = le.transform(df['labels'])

In [ ]:
label_map = dict(zip(le.classes_, le.transform(le.classes_)))
label_inv_map = {v: k for k, v in label_map.items()}

In [ ]:
(train_img, valid_img, train_labels, valid_labels) = train_test_split(df['image'],
                                                                      df['labels'],
                                                                      test_size=0.2,
                                                                      stratify=df['labels'],
                                                                      random_state=RANDOM_SEED)

In [ ]:
train_paths = '../input/resized-plant2021/img_sz_384/' + train_img
valid_paths = '../input/resized-plant2021/img_sz_384/' + valid_img
test_paths = '../input/plant-pathology-2021-fgvc8/test_images/' + sub_df['image']

# Augmentation

In [ ]:
def get_train_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(256,256),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=180, p=0.7),
            albumentations.RandomBrightness(limit=0.6, p=0.5),
            albumentations.Cutout(
                num_holes=8, max_h_size=8, max_w_size=8,
                fill_value=0, always_apply=False, p=0.5
            ),
            albumentations.ShiftScaleRotate(
                shift_limit=0.25, scale_limit=0.1, rotate_limit=0
            ),
            albumentations.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
                max_pixel_value=255.0, always_apply=True
            ),
            ToTensorV2(p=1.0),
        ]
    )

def get_valid_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(256,256),
            albumentations.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
                max_pixel_value=255.0, always_apply=True
            ),
            ToTensorV2(p=1.0)
        ]
    )

# Dataset

In [ ]:
class AppleDataset(Dataset):
    def __init__(self, images_filepaths, labels, transform=None):
        self.images_filepaths = images_filepaths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

In [ ]:
train_dataset = AppleDataset(images_filepaths=train_paths.values,
                             labels=train_labels.values,
                             transform=get_train_transforms())
valid_dataset = AppleDataset(images_filepaths=valid_paths.values,
                             labels=valid_labels.values,
                             transform=get_valid_transforms())

# Metrics

In [ ]:
def multi_acc(output, target):
    y_pred = torch.softmax(output, dim = 1)
    y_pred = torch.argmax(y_pred, dim=1).cpu()
    target = target.cpu()
    
    return accuracy_score(target, y_pred)

In [ ]:
def calculate_f1_macro(output, target):
    y_pred = torch.softmax(output, dim = 1)
    y_pred = torch.argmax(y_pred, dim=1).cpu()
    target = target.cpu()
    
    return f1_score(target, y_pred, average='macro')

In [ ]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"],
                    float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

# Model

In [ ]:
params = {
    'model': 'efficientnet_b3',
    'device': device,
    'lr': 0.001,
    'batch_size': 32,
    'num_workers' : 0,
    'epochs': 7,
    'out_features': df['labels'].nunique()
}

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size=params['batch_size'], shuffle=True,
    num_workers=params['num_workers'], pin_memory=True,
)

val_loader = DataLoader(
    valid_dataset, batch_size=params['batch_size'], shuffle=False,
    num_workers=params['num_workers'], pin_memory=True,
)

In [ ]:
class AppleNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'],
                 pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, out_features)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = AppleNet()
model = model.to(params['device'])
criterion = nn.CrossEntropyLoss().to(params['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])

# Train and Validation

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch, params):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    for i, (images, target) in enumerate(stream, start=1):
        images = images.to(params['device'], non_blocking=True)
        target = target.to(params['device'], non_blocking=True)
        output = model(images)
        loss = criterion(output, target)
        f1_macro = calculate_f1_macro(output, target)
        accuracy = multi_acc(output, target)
        metric_monitor.update('Loss', loss.item())
        metric_monitor.update('F1', f1_macro)
        metric_monitor.update('Accuracy', accuracy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        stream.set_description(
            "Epoch: {epoch}. Train.      {metric_monitor}".format(
                epoch=epoch,
                metric_monitor=metric_monitor)
        )

In [ ]:
def validate(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    with torch.no_grad():
        for i, (images, target) in enumerate(stream, start=1):
            images = images.to(params['device'], non_blocking=True)
            target = target.to(params['device'], non_blocking=True)
            output = model(images)
            loss = criterion(output, target)
            f1_macro = calculate_f1_macro(output, target)
            accuracy = multi_acc(output, target)
            metric_monitor.update('Loss', loss.item())
            metric_monitor.update('F1', f1_macro)
            metric_monitor.update('Accuracy', accuracy)
            stream.set_description(
                "Epoch: {epoch}. Validation. {metric_monitor}".format(
                    epoch=epoch,
                    metric_monitor=metric_monitor)
            )

In [ ]:
for epoch in range(1, params['epochs'] + 1):
    train(train_loader, model, criterion, optimizer, epoch, params)
    validate(val_loader, model, criterion, epoch, params)

## Save Model

In [ ]:
torch.save(model.state_dict(), f"{params['model']}_{params['epochs']}epochs_weights.pth")

# Prediction
I am not actively participating in this competition, thus I am not going to submit this notebook and see my LB score. But anyway this is meant to be a starter Kernel, so little does it matter.  

But I will laydown the foundation for anyone who is interested. So, let's predict and prepare the submission dataframe.

In [ ]:
def multi_acc(output, target):
    y_pred = torch.softmax(output, dim = 1)
    y_pred = torch.argmax(y_pred, dim=1).cpu()
    target = target.cpu()
    
    return accuracy_score(target, y_pred)

In [ ]:
labels = np.zeros(len(test_paths)) # Fake Labels
test_dataset = AppleDataset(images_filepaths=test_paths,
                            labels = labels,
                            transform=get_valid_transforms())
test_loader = DataLoader(
    test_dataset, batch_size=params['batch_size'], shuffle=False,
    num_workers=params['num_workers'], pin_memory=True
)

In [ ]:
model.eval()
predicted_labels = []
with torch.no_grad():
    for (images, target) in test_loader:
        images = images.to(params['device'], non_blocking=True)
        output = model(images)
        predictions = torch.softmax(output, dim = 1)
        predictions = torch.argmax(predictions, dim=1).cpu().numpy()
        predicted_labels += list(predictions)

In [ ]:
sub_df['labels'] = predicted_labels
sub_df['labels'] = sub_df['labels'].map(label_inv_map)

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)

This is a simple starter kernel on implementation of Transfer Learning using Pytorch for this problem.
Pytorch has many SOTA Image models which you can try out using the guidelines in this notebook.

I hope you have learnt something from this notebook. I have created this notebook as a baseline model, which you can easily fork and paly-around with to get much better results. I might update parts of it down the line when I get more GPU hours and some interesting ideas.

**If you liked this notebook and use parts of it in you code, please show some support by upvoting this kernel. It keeps me inspired to come-up with such starter kernels and share it with the community.**

Thanks and happy kaggling!